# Install libraries

In [17]:
import warnings; warnings.simplefilter('ignore')

In [18]:
!pip install selenium
!pip install pymongo

You should consider upgrading via the '/home/guedri/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/guedri/anaconda3/bin/python -m pip install --upgrade pip' command.


# Import packages

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

import pymongo
from pymongo import MongoClient

from bs4 import BeautifulSoup
from time import sleep
import csv
import numpy as np

print('Importing packages (done)')

Importing packages (done)


# Disable Alerts/Notifications on the driver

In [5]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
print('Notifications are disabled')

Notifications are disabled


# Login to Facebook

In [19]:
# Open Chrome and Access Facebook login site
driver = webdriver.Chrome("/usr/bin/chromedriver",chrome_options=chrome_options)
sleep(2)
url = 'https://www.facebook.com'
driver.get(url)
print('Initializing a driver (done)')
sleep(2)

# Import email and password
credential = open('credentials.txt')

line = credential.readlines()
email = line[0]
password = line[1]
print('Importing the login credentials (done)')
sleep(2)

# Input the login credentials
email_field = driver.find_element_by_id('email')
email_field.send_keys(email)
print('Input email (done)')
sleep(3)

password_field = driver.find_element_by_id('pass')
password_field.send_keys(password)
print('Input password (done)')
sleep(3)

print('Login to facebook (done)')

Initializing a driver (done)
Importing the login credentials (done)
Input email (done)
Input password (done)
Login to facebook (done)


# Search for the subject we want to scrape

In [20]:
# Locate the search bar element
search_field = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[2]/div/div/div/div/div/label/input')

# Input the search query to the search bar
search_query = input('What subject do you want to scrape? ')
search_field.send_keys(search_query)

# Search
search_field.send_keys(Keys.RETURN)

print('Search (done)')

What subject do you want to scrape? Elections in france
Search (done)


In [21]:
# Select posts(publications) as a filter of search

current_url = driver.current_url
if len(current_url.split('top'))==2:
    posts_page_url = current_url.split('top')[0]+'posts'+current_url.split('top')[1]
    driver.get(posts_page_url)
sleep(3)
print('Filtering search (done)')

Filtering search (done)


# Scrape the URLs of the posts

In [22]:
# Scroll with infinite loading to the end of the page
last_height = driver.execute_script("return document.body.scrollHeight") # Get scroll height
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(2)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Navigate through the actually page, and extract the URL of each post
page_source = BeautifulSoup(driver.page_source)
urls = page_source.find_all('a')
all_posts_URLs = []
for post in urls:
    post_URL = post.get('href')
    if "https://www.facebook.com/" in post_URL and "/posts/" in post_URL:
        post_URL = post_URL.split('www')[0]+'mbasic'+post_URL.split('www')[1]
        if post_URL not in all_posts_URLs:
            all_posts_URLs.append(post_URL)

print('Scrape URLs (done)')

Scrape URLs (done)


In [23]:
all_posts_URLs

['https://mbasic.facebook.com/MoumniMahmoud/posts/3039083066359358?from_close_friend=1&notif_id=1649282550945232&notif_t=close_friend_activity&ref=notif',
 'https://mbasic.facebook.com/thomasguenoleb/posts/502193831366444?__cft__[0]=AZUvXX-OkmfQ75yxQ-e4Y6k83ubMbVwgFtFbfehpVSMd2gSa1us05-JAawQurpgNSuLB1-93zyvI_zfKCtK0o1JtdpwVotZTtjGFMRRNTD4VWsDPKHVFFVdBf2DknL1lUyA&__tn__=%2CO%2CP-R',
 'https://mbasic.facebook.com/euronews/posts/10159241854133110?__cft__[0]=AZVotcdscEkz1Lz2jXgPm4hsnKQyYdUjlJHjC8vntDIhSLd4POtV3F26qLQxytWpG00JTpOzRY9U_NCvsgZ4z3E4GQsG8iCzDIijF17NBFn9Tli9LkIE4dFctmJIDS2GVB4&__tn__=%2CO%2CP-R',
 'https://mbasic.facebook.com/groups/352181896150872/posts/723129079056150/?__cft__[0]=AZXPTSP8ONLrlNlFIRqj5rPvQJc7obghZjPc2ZyLPBc7K1KDu4Cv00q4wzoavkXDGr6zgOzaYSJqImSB9E--I7gNi6xK2kk5H_Z4FjGdpjJjpMcKMa1VOPXwE3-7E0eDOrPT7feE5eWtslXo3arSDcS4&__tn__=%2CO%2CP-R',
 'https://mbasic.facebook.com/Reuters/posts/5558426110844366?__cft__[0]=AZVZVgMwK0546gHaRYC0Ly56wvHmGcP93YwUp1m-aMSCchsWmaJ16BB9y

# Scrape the data of Posts and save it into MongoDB

In [24]:
# Prepare the database MongoDB

client = pymongo.MongoClient("localhost", 27017)
db = client.fbScraper
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'fbScraper')


In [25]:
def getCommentsElements(comments_bloc):
    divs = comments_bloc.find_elements_by_tag_name("div")
    divs_childs=[]
    dlist=[]
    
    for div in divs:
        divs_childs = div.find_elements_by_tag_name("div")
        if len(divs_childs)>0:
            return divs_childs

In [26]:
# Scrape the data of Posts

posts=[]
comments=[]

for post_URL in all_posts_URLs:
    sleep(3)
    driver.get(post_URL)
    print('Accessing post: ', post_URL)

    sleep(3)
    
    # Extract text from post
    try:
        text = ""
        text_element = driver.find_element_by_xpath("//div[@class='bb']")
        text_childs = text_element.find_elements_by_tag_name("div")
        for child in text_childs:
            text = text + child.text
        post={
            "subject":search_query,
            "text":text,
            "post_url":post_URL
        }
        db["post"].insert_one(post)
    except:
        text = np.nan
    
    # Extract comments
    try:
        comments_id = "ufi_" + post_URL.split('/posts/')[1]
        comments_bloc = driver.find_element_by_id(comments_id)
        
        see_prev_id = "see_prev_" + post_URL.split('/posts/')[1]
        see_next_id = "see_next_" + post_URL.split('/posts/')[1]

        for i in getCommentsElements(comments_bloc):
            divs = i.find_elements_by_tag_name("div")
            if len(divs)>0:
                for div in divs:
                    dlist = div.find_elements_by_tag_name("div")
                    for d in dlist:
                        if len(d.get_attribute("class").split(" ")) == 1:
                            comment = {
                            "subject": search_query,
                            "post_url": post_URL,
                            "text": d.text
                            }
                            db["comment"].insert_one(comment)
    except:
        pass
        
    sleep(3)
    driver.execute_script("window.scrollTo(0, 1000)")
        
    sleep(5)
print("Scraping (done)")

Accessing post:  https://mbasic.facebook.com/MoumniMahmoud/posts/3039083066359358?from_close_friend=1&notif_id=1649282550945232&notif_t=close_friend_activity&ref=notif
Accessing post:  https://mbasic.facebook.com/thomasguenoleb/posts/502193831366444?__cft__[0]=AZUvXX-OkmfQ75yxQ-e4Y6k83ubMbVwgFtFbfehpVSMd2gSa1us05-JAawQurpgNSuLB1-93zyvI_zfKCtK0o1JtdpwVotZTtjGFMRRNTD4VWsDPKHVFFVdBf2DknL1lUyA&__tn__=%2CO%2CP-R
Accessing post:  https://mbasic.facebook.com/euronews/posts/10159241854133110?__cft__[0]=AZVotcdscEkz1Lz2jXgPm4hsnKQyYdUjlJHjC8vntDIhSLd4POtV3F26qLQxytWpG00JTpOzRY9U_NCvsgZ4z3E4GQsG8iCzDIijF17NBFn9Tli9LkIE4dFctmJIDS2GVB4&__tn__=%2CO%2CP-R
Accessing post:  https://mbasic.facebook.com/groups/352181896150872/posts/723129079056150/?__cft__[0]=AZXPTSP8ONLrlNlFIRqj5rPvQJc7obghZjPc2ZyLPBc7K1KDu4Cv00q4wzoavkXDGr6zgOzaYSJqImSB9E--I7gNi6xK2kk5H_Z4FjGdpjJjpMcKMa1VOPXwE3-7E0eDOrPT7feE5eWtslXo3arSDcS4&__tn__=%2CO%2CP-R
Accessing post:  https://mbasic.facebook.com/Reuters/posts/5558426110844366?_

# Get the data from MongoDB

In [27]:
for post_element in db["post"].find():
    print(post_element)

{'_id': ObjectId('62506d741a08b23f57b14264'), 'subject': 'le décès du président Jacques Chirac', 'text': '😞❤️😞❤️', 'post_url': 'https://mbasic.facebook.com/MoumniMahmoud/posts/3039083066359358?from_close_friend=1&notif_id=1649282550945232&notif_t=close_friend_activity&ref=notif'}
{'_id': ObjectId('62506d851a08b23f57b14265'), 'subject': 'le décès du président Jacques Chirac', 'text': 'Afficher l’historique des modificationsCharles Millon, ancien ministre de la Défense du président Jacques Chirac, allume Alassane Ouattara et Jean-Yves Le Drian dans mondafrique.com sur le MALI 🇲🇱\n🔸Mondafrique: Le président ivoirien Ouattara a déclaré jeudi matin sur RFI et France 24 qu’il demandait « à ses frères maliens de faire un effort, de rentrer dans les rangs ». Est ce que vous lui donnez raison ?\n🔹Charles Millon: Il faudrait rappeler à Monsieur Ouattara qui remet en cause la légitimité de la junte militaire à Bamako qu’il bénéficie d’un troisième mandat illégitime et non constitutionnel. Qu’il n